<a href="https://colab.research.google.com/github/41monster/RePIT_NaturalCirculation2D/blob/main/Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
######################################################################################
# The current version (v.1) has a lot of hardcoding, and we are upgrading our codes. #
######################################################################################

# Please run this code after Pretraining.ipynb running (if not, you should import functions in Pretraining.ipynb)
# Part 1. Prediction instruction
st = 10.02 # starting time [s]
dt = 0.01 # timestep [s]
nt = 100 # number of timestep [-]
model.load_weights(checkpoint_path) #pre-trainied weights

In [ ]:
# Part 2. Data pre-processing and prediction
for nti in range(0,nt+1):
    tnow = st + dt*nti
    tnow_f = st + dt*nti
    tnow = "{:g}".format(tnow)

    if nti == 0: # for first prediction, previous OpenFOAM time series data is input
        # OpenFOAM time series data load
        filename_openfoam = '/home/ubuntu/ML_NC/traindata%s.csv'%tnow # you should change the directory

        openfoam = pd.read_csv(filename_openfoam, header = None)
        openfoam_slice = openfoam.drop(0, axis=0).astype(np.float32)
        openfoam_tf = tf.reshape(openfoam_slice, shape =[-1,7])
        openfoam_np = openfoam_tf.numpy()

        alpha_np = openfoam_np[:,1:2]
        p_np = openfoam_np[:,2:3]
        ph_np = openfoam_np[:,3:4]
        U_np = openfoam_np[:,4:6]

        # domain matrix
        alpha_matrix = np.ones([200,200])
        p_matrix = np.ones([200,200])
        ph_matrix = np.ones([200,200])
        ux_matrix = np.ones([200,200])
        uy_matrix = np.ones([200,200])

        for i in range(0, grid_number):
            ngrid_x = Ngrid_xnp[i]
            ngrid_y = Ngrid_ynp[i]
            alpha_matrix[ngrid_x-1,ngrid_y-1] = alpha_np[i,0]
            p_matrix[ngrid_x-1,ngrid_y-1] = p_np[i,0]
            ph_matrix[ngrid_x-1,ngrid_y-1] = ph_np[i,0]
            ux_matrix[ngrid_x-1,ngrid_y-1] = U_np[i,0]
            uy_matrix[ngrid_x-1,ngrid_y-1] = U_np[i,1]

        # saving domain matrix
        globals()['alpha_matrix_true_'+str(tnow)] = copy(alpha_matrix)
        globals()['p_matrix_true_'+str(tnow)] = copy(p_matrix)  # global variable
        globals()['ph_matrix_true_'+str(tnow)] = copy(ph_matrix)
        globals()['ux_matrix_true_'+str(tnow)] = copy(ux_matrix)
        globals()['uy_matrix_true_'+str(tnow)] = copy(uy_matrix)

        rho_matrix = 348.33/alpha_matrix # perfect ideal gas law
        globals()['rho_matrix_true_'+str(tnow)] = copy(rho_matrix)

        # ML dataset production
        ngrid_x_zp = grid_number_x + 2; ngrid_y_zp = grid_number_y +2

        p_ml = np.ones([grid_number, 5]); ph_ml = np.ones([grid_number, 5]); ux_ml = np.ones([grid_number, 5]); uy_ml = np.ones([grid_number, 5]); alpha_ml = np.ones([grid_number, 5])
        p_matrix_zp = np.zeros([ngrid_x_zp,ngrid_y_zp]); ph_matrix_zp = np.zeros([ngrid_x_zp,ngrid_y_zp]); ux_matrix_zp = np.zeros([ngrid_x_zp,ngrid_y_zp]); uy_matrix_zp = np.zeros([ngrid_x_zp,ngrid_y_zp]); alpha_matrix_zp = np.zeros([ngrid_x_zp,ngrid_y_zp]) # zero padding
        p_matrix_zp[1:ngrid_x_zp-1, 1:ngrid_y_zp-1] = p_matrix; ph_matrix_zp[1:ngrid_x_zp-1, 1:ngrid_y_zp-1] = ph_matrix; ux_matrix_zp[1:ngrid_x_zp-1, 1:ngrid_y_zp-1] = ux_matrix; uy_matrix_zp[1:ngrid_x_zp-1, 1:ngrid_y_zp-1] = uy_matrix; alpha_matrix_zp[1:ngrid_x_zp-1, 1:ngrid_y_zp-1] = alpha_matrix

        k = 0
        for i in range(2, 200):  # except 1-layer boundary cells
            for j in range(2,200):
                p_ml[k,0] = p_matrix_zp[i,j]; p_ml[k,1] = p_matrix_zp[i-1,j]; p_ml[k,2] = p_matrix_zp[i+1,j]; p_ml[k,3] = p_matrix_zp[i,j-1]; p_ml[k,4] = p_matrix_zp[i,j+1];
                ph_ml[k,0] = ph_matrix_zp[i,j]; ph_ml[k,1] = ph_matrix_zp[i-1,j]; ph_ml[k,2] = ph_matrix_zp[i+1,j]; ph_ml[k,3] = ph_matrix_zp[i,j-1]; ph_ml[k,4] = ph_matrix_zp[i,j+1];
                ux_ml[k,0] = ux_matrix_zp[i,j]; ux_ml[k,1] = ux_matrix_zp[i-1,j]; ux_ml[k,2] = ux_matrix_zp[i+1,j]; ux_ml[k,3] = ux_matrix_zp[i,j-1]; ux_ml[k,4] = ux_matrix_zp[i,j+1];
                uy_ml[k,0] = uy_matrix_zp[i,j]; uy_ml[k,1] = uy_matrix_zp[i-1,j]; uy_ml[k,2] = uy_matrix_zp[i+1,j]; uy_ml[k,3] = uy_matrix_zp[i,j-1]; uy_ml[k,4] = uy_matrix_zp[i,j+1];
                alpha_ml[k,0] = alpha_matrix_zp[i,j]; alpha_ml[k,1] = alpha_matrix_zp[i-1,j]; alpha_ml[k,2] = alpha_matrix_zp[i+1,j]; alpha_ml[k,3] = alpha_matrix_zp[i,j-1]; alpha_ml[k,4] = alpha_matrix_zp[i,j+1];
                k=k+1

        ml_dataset_pre = np.concatenate((alpha_ml,ux_ml,uy_ml),axis = 1)

    else: # for first prediction, previous ML time series data is input

        filename_openfoam = '/home/ubuntu/ML_NC/traindata%s.csv'%tnow # for accuracy check (not for ML input)

        openfoam = pd.read_csv(filename_openfoam, header = None)
        openfoam_slice = openfoam.drop(0, axis=0).astype(np.float32)
        openfoam_tf = tf.reshape(openfoam_slice, shape =[-1,7])
        openfoam_np = openfoam_tf.numpy()

        alpha_np = openfoam_np[:,1:2]
        p_np = openfoam_np[:,2:3]
        ph_np = openfoam_np[:,3:4]
        U_np = openfoam_np[:,4:6]

        # domain matrix
        alpha_matrix = np.ones([200,200])
        p_matrix = np.ones([200,200])
        ph_matrix = np.ones([200,200])
        ux_matrix = np.ones([200,200])
        uy_matrix = np.ones([200,200])

        for i in range(0, grid_number):
            ngrid_x = Ngrid_xnp[i]
            ngrid_y = Ngrid_ynp[i]
            alpha_matrix[ngrid_x-1,ngrid_y-1] = alpha_np[i,0]
            p_matrix[ngrid_x-1,ngrid_y-1] = p_np[i,0]
            ph_matrix[ngrid_x-1,ngrid_y-1] = ph_np[i,0]
            ux_matrix[ngrid_x-1,ngrid_y-1] = U_np[i,0]
            uy_matrix[ngrid_x-1,ngrid_y-1] = U_np[i,1]

        globals()['alpha_matrix_true_'+str(tnow)] = copy(alpha_matrix)
        globals()['p_matrix_true_'+str(tnow)] = copy(p_matrix)   # global variable
        globals()['ph_matrix_true_'+str(tnow)] = copy(ph_matrix)
        globals()['ux_matrix_true_'+str(tnow)] = copy(ux_matrix)
        globals()['uy_matrix_true_'+str(tnow)] = copy(uy_matrix)

        # Residual check for ground truth (for comparison)
        # mass
        pinn_dt = np.ones([198,198])
        pinn_dudx = np.ones([198,198])
        pinn_dvdy = np.ones([198,198])

        rho_matrix = 348.33/alpha_matrix   # perfect ideal gas law:100000*28.96/8.314/1000,1.1839/(2-293/303)
        globals()['rho_matrix_true_'+str(tnow)] = copy(rho_matrix)

        tpre = tnow_f-dt
        tpre = "{:g}".format(tpre)

        pinn_dudx = (ux_matrix[2:200,1:199] - ux_matrix[0:198,1:199])/(2*0.005)
        pinn_dvdy = (uy_matrix[1:199,2:200] - uy_matrix[1:199,0:198])/(2*0.005)

        Rs_mass = pinn_dudx+pinn_dvdy
        Rs_mass_sq = Rs_mass*Rs_mass
        Rs_mass_sum = Rs_mass_sq.sum()/40000
        globals()['Rs_mass_true_sum'+str(tnow)] = Rs_mass_sum

        # momentum
        y_mom_1 = (globals()['ux_matrix_true_'+str(tnow)][1:199,1:199] - globals()['ux_matrix_true_'+str(tpre)][1:199,1:199])
        y_mom_3_1 = ux_matrix[1:199,1:199]
        y_mom_3_2 = ux_matrix[2:200,1:199] - ux_matrix[0:198,1:199]
        y_mom_3 = y_mom_3_1*y_mom_3_2
        y_mom_4_1 = uy_matrix[1:199,1:199]
        y_mom_4_2 = ux_matrix[1:199,2:200] - ux_matrix[1:199,0:198]
        y_mom_4 = y_mom_4_1*y_mom_4_2
        y_mom_5_2 = ux_matrix[1:199,2:200] - 2*ux_matrix[1:199,1:199] + ux_matrix[1:199,0:198]

        y_mom_5 = 1.831e-05/rho_matrix[1:199,1:199]*(y_mom_5_2)
        y_mom_6 = 9.81/293*(293*np.ones([198,198])-alpha_matrix[1:199,1:199])

        Rs_mom = y_mom_1/0.01 +  y_mom_3/(2*0.005) + y_mom_4/(2*0.005) - y_mom_5/(0.005*0.005) - y_mom_6


        Rs_mom_sq = Rs_mom*Rs_mom
        Rs_mom_sum = Rs_mom_sq.sum()/40000
        globals()['Rs_mom_true_sum'+str(tnow)] = Rs_mom_sum

        # heaT
        tdiff_matrix = (0.14*(alpha_matrix[1:199,1:199] - 293*np.ones([198,198]))+ 21.7)/1000000
        y_heat_1 = (globals()['alpha_matrix_true_'+str(tnow)][1:199,1:199] - globals()['alpha_matrix_true_'+str(tpre)][1:199,1:199])
        y_heat_2_1 = (alpha_matrix[2:200,1:199] - alpha_matrix[0:198,1:199])
        y_heat_2_2 = ux_matrix[1:199,1:199]
        y_heat_2 = y_heat_2_1*y_heat_2_2
        y_heat_3_1 = (alpha_matrix[1:199,2:200] - alpha_matrix[1:199,0:198])
        y_heat_3_2 = uy_matrix[1:199,1:199]
        y_heat_3 = y_heat_3_1*y_heat_3_2
        y_heat_4 = tdiff_matrix*(alpha_matrix[1:199,2:200] - 2*alpha_matrix[1:199,1:199] + alpha_matrix[1:199,0:198])

        Rs_heat = y_heat_1/0.01 + y_heat_2/(2*0.005) + y_heat_3/(2*0.005) - y_heat_4/(0.005*0.005)

        Rs_heat_sq = Rs_heat*Rs_heat
        Rs_heat_sum = Rs_heat_sq.sum()/40000
        globals()['Rs_heat_true_sum'+str(tnow)] = Rs_heat_sum


        k = 0
        for i in range(1, 199):
            for j in range(1,199):
                alpha_matrix[i,j] = a_new[k,0]
                #ph_matrix[i,j] = ph_new[k,0]
                ux_matrix[i,j] = ux_new[k,0] #ux_new[k,0]
                uy_matrix[i,j] = uy_new[k,0]
                k = k+1

        # ML prediction field
        globals()['a_matrix_pred_'+str(tnow)] =  copy(alpha_matrix)
        globals()['ph_matrix_pred_'+str(tnow)] = copy(ph_matrix)
        globals()['ux_matrix_pred_'+str(tnow)] = copy(ux_matrix)
        globals()['uy_matrix_pred_'+str(tnow)] = copy(uy_matrix)


        if nti >= 2:
            # Residual check for ML predicted field
            # mass
            pinn_dt = np.ones([198,198])
            pinn_dudx = np.ones([198,198])
            pinn_dvdy = np.ones([198,198])

            rho_matrix = 348.33/alpha_matrix   #100000*28.96/8.314/1000
            globals()['rho_matrix_pred_'+str(tnow)] = copy(rho_matrix)

            tpre = tnow_f-dt
            tpre = "{:g}".format(tpre)

            pinn_dudx = (ux_matrix[2:200,1:199] - ux_matrix[0:198,1:199])/(2*0.005)
            pinn_dvdy = (uy_matrix[1:199,2:200] - uy_matrix[1:199,0:198])/(2*0.005)

            Rs_mass = pinn_dudx+pinn_dvdy
            Rs_mass_sq = Rs_mass*Rs_mass
            Rs_mass_sum = Rs_mass_sq.sum()/40000
            globals()['Rs_mass_pred_sum'+str(tnow)] = Rs_mass_sum

            if Rs_mass_sum >= 0.001:  # residual criterion in this study
                print('mass residual issue!')
                saved_nti = nti
                break

            # momentum
            y_mom_1 = (globals()['ux_matrix_pred_'+str(tnow)][1:199,1:199] - globals()['ux_matrix_pred_'+str(tpre)][1:199,1:199])
            y_mom_3_1 = ux_matrix[1:199,1:199]
            y_mom_3_2 = ux_matrix[2:200,1:199] - ux_matrix[0:198,1:199]
            y_mom_3 = y_mom_3_1*y_mom_3_2
            y_mom_4_1 = uy_matrix[1:199,1:199]
            y_mom_4_2 = ux_matrix[1:199,2:200] - ux_matrix[1:199,0:198]
            y_mom_4 = y_mom_4_1*y_mom_4_2
            y_mom_5_2 = ux_matrix[1:199,2:200] - 2*ux_matrix[1:199,1:199] + ux_matrix[1:199,0:198]

            y_mom_5 = 1.831e-05/rho_matrix[1:199,1:199]*(y_mom_5_2)
            y_mom_6 = 9.81/293*(293*np.ones([198,198])-alpha_matrix[1:199,1:199])

            Rs_mom = y_mom_1/0.01 +  y_mom_3/(2*0.005) + y_mom_4/(2*0.005) - y_mom_5/(0.005*0.005) - y_mom_6
            Rs_mom_sq = Rs_mom*Rs_mom
            Rs_mom_sum = Rs_mom_sq.sum()/40000
            globals()['Rs_mom_pred_sum'+str(tnow)] = Rs_mom_sum

            # heat
            tdiff_matrix = (0.14*(alpha_matrix[1:199,1:199] - 293*np.ones([198,198]))+ 21.7)/1000000
            y_heat_1 = (globals()['a_matrix_pred_'+str(tnow)][1:199,1:199] - globals()['a_matrix_pred_'+str(tpre)][1:199,1:199])
            y_heat_2_1 = (alpha_matrix[2:200,1:199] - alpha_matrix[0:198,1:199])
            y_heat_2_2 = ux_matrix[1:199,1:199]
            y_heat_2 = y_heat_2_1*y_heat_2_2
            y_heat_3_1 = (alpha_matrix[1:199,2:200] - alpha_matrix[1:199,0:198])
            y_heat_3_2 = uy_matrix[1:199,1:199]
            y_heat_3 = y_heat_3_1*y_heat_3_2
            y_heat_4 = tdiff_matrix*(alpha_matrix[1:199,2:200] - 2*alpha_matrix[1:199,1:199] + alpha_matrix[1:199,0:198])

            Rs_heat = y_heat_1/0.01 + y_heat_2/(2*0.005) + y_heat_3/(2*0.005) - y_heat_4/(0.005*0.005)

            Rs_heat_sq = Rs_heat*Rs_heat
            Rs_heat_sum = Rs_heat_sq.sum()/40000
            globals()['Rs_heat_pred_sum'+str(tnow)] = Rs_heat_sum

        a_matrix_new = copy(alpha_matrix)
        ph_matrix_new = copy(ph_matrix)
        ux_matrix_new = copy(ux_matrix)
        uy_matrix_new = copy(uy_matrix)

        globals()['a_matrix_error_'+str(tnow)] = abs(globals()['alpha_matrix_true_'+str(tnow)] - globals()['a_matrix_pred_'+str(tnow)])/abs(globals()['alpha_matrix_true_'+str(tnow)])*100
        globals()['ux_matrix_error_'+str(tnow)] = abs(globals()['ux_matrix_true_'+str(tnow)] - globals()['ux_matrix_pred_'+str(tnow)])
        globals()['uy_matrix_error_'+str(tnow)] = abs(globals()['uy_matrix_true_'+str(tnow)] - globals()['uy_matrix_pred_'+str(tnow)])

        print(tnow) # check the prediction status

        # continous ML prediction ml_t -> ml_t+1
        p_ml_new = np.ones([grid_number, 5]); ph_ml_new = np.ones([grid_number, 5]); ux_ml_new = np.ones([grid_number, 5]); uy_ml_new = np.ones([grid_number, 5]); alpha_ml_new = np.ones([grid_number, 5])
        p_matrix_zp = np.zeros([ngrid_x_zp,ngrid_y_zp]); ph_matrix_zp = np.zeros([ngrid_x_zp,ngrid_y_zp]); ux_matrix_zp = np.zeros([ngrid_x_zp,ngrid_y_zp]); uy_matrix_zp = np.zeros([ngrid_x_zp,ngrid_y_zp]); alpha_matrix_zp = np.zeros([ngrid_x_zp,ngrid_y_zp]) # zero padding
        p_matrix_zp[1:ngrid_x_zp-1, 1:ngrid_y_zp-1] = p_matrix; ph_matrix_zp[1:ngrid_x_zp-1, 1:ngrid_y_zp-1] = ph_matrix_new; ux_matrix_zp[1:ngrid_x_zp-1, 1:ngrid_y_zp-1] = ux_matrix_new; uy_matrix_zp[1:ngrid_x_zp-1, 1:ngrid_y_zp-1] = uy_matrix_new; alpha_matrix_zp[1:ngrid_x_zp-1, 1:ngrid_y_zp-1] = a_matrix_new

        k = 0
        for i in range(2, 200):   # except 1-layer boundary cells
            for j in range(2,200):  # except 1-layer boundary cells
                p_ml_new[k,0] = p_matrix_zp[i,j]; p_ml_new[k,1] = p_matrix_zp[i-1,j]; p_ml_new[k,2] = p_matrix_zp[i+1,j]; p_ml_new[k,3] = p_matrix_zp[i,j-1]; p_ml_new[k,4] = p_matrix_zp[i,j+1];
                ph_ml_new[k,0] = ph_matrix_zp[i,j]; ph_ml_new[k,1] = ph_matrix_zp[i-1,j]; ph_ml_new[k,2] = ph_matrix_zp[i+1,j]; ph_ml_new[k,3] = ph_matrix_zp[i,j-1]; ph_ml_new[k,4] = ph_matrix_zp[i,j+1];
                ux_ml_new[k,0] = ux_matrix_zp[i,j]; ux_ml_new[k,1] = ux_matrix_zp[i-1,j]; ux_ml_new[k,2] = ux_matrix_zp[i+1,j]; ux_ml_new[k,3] = ux_matrix_zp[i,j-1]; ux_ml_new[k,4] = ux_matrix_zp[i,j+1];
                uy_ml_new[k,0] = uy_matrix_zp[i,j]; uy_ml_new[k,1] = uy_matrix_zp[i-1,j]; uy_ml_new[k,2] = uy_matrix_zp[i+1,j]; uy_ml_new[k,3] = uy_matrix_zp[i,j-1]; uy_ml_new[k,4] = uy_matrix_zp[i,j+1];
                alpha_ml_new[k,0] = alpha_matrix_zp[i,j]; alpha_ml_new[k,1] = alpha_matrix_zp[i-1,j]; alpha_ml_new[k,2] = alpha_matrix_zp[i+1,j]; alpha_ml_new[k,3] = alpha_matrix_zp[i,j-1]; alpha_ml_new[k,4] = alpha_matrix_zp[i,j+1];
                k=k+1


        ml_dataset_pre  = np.concatenate((alpha_ml_new,ux_ml_new,uy_ml_new),axis = 1)

    ml_dataset_pd = pd.DataFrame(ml_dataset_pre)
    normed_pred_data = norm(ml_dataset_pd)
    normed_pred_data_re= tf.reshape(normed_pred_data, shape=[-1,15])

    x_pred = normed_pred_data_re

    #model.load_weights(checkpoint_path)
    train_predictions_a = model.net_a(x_pred)
    train_predictions_ph = model.net_ph(x_pred)
    train_predictions_ux = model.net_u(x_pred)
    train_predictions_uy = model.net_v(x_pred)

    train_predictions_tot = tf.concat([train_predictions_a, train_predictions_ux, train_predictions_uy], axis = 1)
    train_predictions_pd = pd.DataFrame(train_predictions_tot)

    denorm_predictions_tot = denorm_label(train_predictions_pd)
    denorm_predictions_tot_re = tf.reshape(denorm_predictions_tot, shape = [-1,3])

    denorm_train_predictions_a = np.reshape(denorm_predictions_tot_re[:, 0:1], (-1,1))
    denorm_train_predictions_ux = np.reshape(denorm_predictions_tot_re[:, 1:2], (-1,1))
    denorm_train_predictions_uy = np.reshape(denorm_predictions_tot_re[:, 2:3], (-1,1))

    a_new = ml_dataset_pre[:, 0:1] + denorm_train_predictions_a
    ux_new = ml_dataset_pre[:, 5:6] + denorm_train_predictions_ux
    uy_new = ml_dataset_pre[:, 10:11] + denorm_train_predictions_uy

In [ ]:
# Part 3. Residual check
Rs_mass_true_history = []
Rs_mass_pred_history = []
Rs_mom_true_history = []
Rs_mom_pred_history = []
Rs_heat_true_history = []
Rs_heat_pred_history = []
a_error_history = []

for nti in range(2,saved_nti):
    tnow = st + dt*nti
    tnow = "{:g}".format(tnow)
    a_error_history.append(np.max(globals()['a_matrix_error_'+str(tnow)]))
    Rs_mass_true_history.append(globals()['Rs_mass_true_sum'+str(tnow)])
    Rs_mass_pred_history.append(globals()['Rs_mass_pred_sum'+str(tnow)])
    Rs_mom_true_history.append(globals()['Rs_mom_true_sum'+str(tnow)])
    Rs_mom_pred_history.append(globals()['Rs_mom_pred_sum'+str(tnow)])
    Rs_heat_true_history.append(globals()['Rs_heat_true_sum'+str(tnow)])
    Rs_heat_pred_history.append(globals()['Rs_heat_pred_sum'+str(tnow)])

# residual
x = np.arange(2,saved_nti)
plt.plot(x,Rs_mass_true_history,':k', label = 'OpenFOAM')
plt.plot(x,Rs_mass_pred_history, '-b', label = 'FVMN')
plt.ylim(0,0.001)
plt.xlim(1, saved_nti)
plt.ylabel('Residual')
plt.xlabel('# of timesteps')
plt.legend()


plt.show

# relaive error
x = np.arange(2,saved_nti)
plt.plot(x,a_error_history,'-r', label = 'FVMN')
#plt.ylim(0,0.5)
plt.xlim(1, saved_nti)
plt.ylabel('Maximum relative error')
plt.xlabel('# of timesteps')
plt.legend()


plt.show

In [ ]:
# Part 4. ML results save for OpenFoam re-running
saved_tnow = st + dt*(saved_nti-2)
saved_tnow = "{:g}".format(saved_tnow)

a_results_excel = np.zeros([40000,1])
ux_results_excel = np.zeros([40000,1])
uy_results_excel = np.zeros([40000,1])

k3 = 0
for j in range(0,200):
    for i in range(0,200):
        a_results_excel[k3,0] = globals()['a_matrix_pred_'+str(saved_tnow)][i,j]
        ux_results_excel[k3,0] = globals()['ux_matrix_pred_'+str(saved_tnow)][i,j]
        uy_results_excel[k3,0] = globals()['uy_matrix_pred_'+str(saved_tnow)][i,j]

        k3 = k3 + 1

results_excel = np.concatenate((a_results_excel, ux_results_excel, uy_results_excel),axis = 1)
results_true_save = pd.DataFrame(results_excel)
results_true_save.to_excel(excel_writer='/home/ubuntu/ML_NC/resultdata_%s.xlsx'%saved_tnow)

In [ ]:
# Part 5. ML results save for post-processing
a_results_excel = np.zeros([40000,1])
ux_results_excel = np.zeros([40000,1])
uy_results_excel = np.zeros([40000,1])

k3 = 0
for j in range(0,200):
    for i in range(0,200):
        a_results_excel[k3,0] = globals()['a_matrix_pred_'+str(10.5)][i,j] #hardcoding
        ux_results_excel[k3,0] = globals()['ux_matrix_pred_'+str(10.5)][i,j]
        uy_results_excel[k3,0] = globals()['uy_matrix_pred_'+str(10.5)][i,j]

        k3 = k3 + 1

results_excel = np.concatenate((a_results_excel, ux_results_excel, uy_results_excel),axis = 1)
results_true_save = pd.DataFrame(results_excel)
results_true_save.to_excel(excel_writer='/home/ubuntu/ML_NC/resultdata_1050.xlsx')